In [ ]:
import json
import sqlite3 as lite

import igraph
import networkx as nx

%matplotlib inline
import matplotlib.pyplot as plt

## Grab data
We really don't have that much data, so just load it all into memory

In [ ]:
with lite.connect('database.db') as con:
    cur = con.cursor()
    
    cur.execute('select band_id,band from Bands where band_id in (select band_id from Similarities)')
    bands_list = cur.fetchall()
    
    cur.execute('select band_id,similar_to_id,score from Similarities where similar_to_id in (select band_id from Similarities)')
    sim_list = cur.fetchall()

In [ ]:
num_nodes = len(bands_list)
num_edges = len(sim_list)
print(num_nodes,num_edges)

In [ ]:
band_id_to_band = {band_id:band for band_id,band in bands_list}
band_to_band_id = {band:band_id for band_id,band in bands_list}

In [ ]:
print(bands_list[0:10])
print(sim_list[0:10])
node_list = list(set(band_id for band_id,_,_ in sim_list))
edge_list = [(band_id,similar_to_id,score) for band_id,similar_to_id,score in sim_list]
print(edge_list[0:10])

## Kamada-Kawai with `nx`

In [ ]:
G = nx.DiGraph()
G.add_weighted_edges_from(edge_list[0:50])
G = nx.relabel_nodes(G, band_id_to_band)

In [ ]:
pos = nx.kamada_kawai_layout(G)
#pos = nx.nx_agraph.graphviz_layout(G)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9));
nx.draw_networkx(G, pos=pos)

## Kamada-Kawai with `igraph`

In [ ]:
band_id_to_igraph_node = {band_id:i for i,band_id in enumerate(band_id_to_band.keys())}

In [ ]:
G = igraph.Graph()
G.add_vertices(map(lambda band_id: band_id_to_igraph_node[band_id], node_list))
G.add_edges(map(lambda t: (band_id_to_igraph_node[t[0]],band_id_to_igraph_node[t[1]]), edge_list))

In [ ]:
print(node_list[0:2])
print(edge_list[0:2])

## D3.js?
Well, I didn't really give them a fair chance, but if it's not interactive...
Let's try getting D3 inside the notebook.
https://ipython-books.github.io/64-visualizing-a-networkx-graph-in-the-notebook-with-d3js/

In [ ]:
G = nx.DiGraph()
G.add_weighted_edges_from(edge_list[:25000])
G = nx.relabel_nodes(G, band_id_to_band)

In [ ]:
# https://stackoverflow.com/questions/8576737/how-to-only-keep-nodes-in-networkx-graph-with-2-outgoing-edges-or-0-outgoing-ed
outdeg = G.out_degree()
keep = [n for (n,deg) in outdeg if outdeg[n] > 1]
G = G.subgraph(keep)

In [ ]:
#fig, ax = plt.subplots(1, 1, figsize=(8, 6));
#nx.draw_networkx(G, ax=ax)

In [ ]:
print(list(G.nodes())[0:10])
nodes = [{'name': str(i)} for i in G.nodes()]
links = [{'source': u[0], 'target': u[1]}
         for u in G.edges()]
with open('graph.json', 'w') as f:
    json.dump({'nodes': nodes, 'links': links},
              f, indent=4,)

In [ ]:
%%html
<div id="d3-example"></div>
<style>
.node {stroke: #fff; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}
</style>

In [ ]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths:
    {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
  // The code in this block is executed when the
  // d3.js library has been loaded.

  // First, we specify the size of the canvas
  // containing the visualization (size of the
  // <div> element).
  var width = 300, height = 300;

  // We create a color scale.
  var color = d3.scale.category10();

  // We create a force-directed dynamic graph layout.
  var force = d3.layout.force()
    .charge(-120)
    .linkDistance(30)
    .size([width, height]);

  // In the <div> element, we create a <svg> graphic
  // that will contain our interactive visualization.
  var svg = d3.select("#d3-example").select("svg")
  if (svg.empty()) {
    svg = d3.select("#d3-example").append("svg")
          .attr("width", width)
          .attr("height", height);
  }

  // We load the JSON file.
  d3.json("graph.json", function(error, graph) {
    // In this block, the file has been loaded
    // and the 'graph' object contains our graph.

    // We load the nodes and links in the
    // force-directed graph.
    force.nodes(graph.nodes)
      .links(graph.links)
      .start();

    // We create a <line> SVG element for each link
    // in the graph.
    var link = svg.selectAll(".link")
      .data(graph.links)
      .enter().append("line")
      .attr("class", "link");

    // We create a <circle> SVG element for each node
    // in the graph, and we specify a few attributes.
    var node = svg.selectAll(".node")
      .data(graph.nodes)
      .enter().append("circle")
      .attr("class", "node")
      .attr("r", 5)  // radius
      .call(force.drag);

    // The name of each node is the node number.
    node.append("title")
        .text(function(d) { return d.name; });

    // We bind the positions of the SVG elements
    // to the positions of the dynamic force-directed
    // graph, at each time step.
    force.on("tick", function() {
      link.attr("x1", function(d){return d.source.x})
          .attr("y1", function(d){return d.source.y})
          .attr("x2", function(d){return d.target.x})
          .attr("y2", function(d){return d.target.y});

      node.attr("cx", function(d){return d.x})
          .attr("cy", function(d){return d.y});
    });
  });
});

In [ ]:
%%html
<meta charset="utf-8">
<style>

.links line {
  stroke: #999;
  stroke-opacity: 0.6;
}

.nodes circle {
  stroke: #fff;
  stroke-width: 1.5px;
}

text {
  font-family: sans-serif;
  font-size: 10px;
}

</style>
<svg width="960" height="600" id="d3ex2"></svg>
<script src="https://d3js.org/d3.v4.min.js"></script>

In [ ]:
%%html
<meta charset="utf-8">
<style>

.links line {
  stroke: #999;
  stroke-opacity: 0.6;
}

.nodes circle {
  stroke: #fff;
  stroke-width: 1.5px;
}

text {
  font-family: sans-serif;
  font-size: 10px;
}

</style>
<svg width="960" height="600" id="d3ex2"></svg>

## Okay, the graph is too big for direct visualization
How about picking a band and showing the subgraph that's just k hops away from that band?
Sounds like this is called the ego subgraph.

In [ ]:
band='Panopticon'

G = nx.DiGraph()
G.add_weighted_edges_from(edge_list)
G = nx.relabel_nodes(G, band_id_to_band)

# https://stackoverflow.com/questions/17301887/how-to-compute-nearby-nodes-with-networkx
ego = nx.ego_graph(G, 'Panopticon', radius=2, center=True)

len(ego.nodes())

In [ ]:
#fig, ax = plt.subplots(1, 1, figsize=(8, 6));
#nx.draw_networkx(ego, ax=ax)

In [ ]:
nodes = [{'name': str(i)} for i in ego.nodes()]
links = [{'source': u[0], 'target': u[1]}
         for u in ego.edges()]
with open('ego.json', 'w') as f:
    json.dump({'nodes': nodes, 'links': links},
              f, indent=4,)

In [ ]:
%%html
<meta charset="utf-8">
<style>

.links line {
  stroke: #999;
  stroke-opacity: 0.6;
}

.nodes circle {
  stroke: #fff;
  stroke-width: 1.5px;
}

text {
  font-family: sans-serif;
  font-size: 10px;
}

</style>
<svg width="960" height="600" id="d3ego"></svg>

In [ ]:
%%javascript
require.config({paths:
    {d3: "http://d3js.org/d3.v4.min"}});
require(["d3"], function(d3) {

var svg = d3.select("#d3ego"),
    width = +svg.attr("width"),
    height = +svg.attr("height");

//var color = d3.scaleOrdinal(d3.schemeCategory20);

var simulation = d3.forceSimulation()
    .force("link", d3.forceLink().id(function(d) { return d.name; }))
    .force("charge", d3.forceManyBody())
    .force("center", d3.forceCenter(width / 2, height / 2));

d3.json("ego.json", function(error, graph) {
  if (error) throw error;

  var link = svg.append("g")
      .attr("class", "links")
    .selectAll("line")
    .data(graph.links)
    .enter().append("line");
      //.attr("stroke-width", function(d) { return Math.sqrt(d.value); });

  var node = svg.append("g")
      .attr("class", "nodes")
    .selectAll("g")
    .data(graph.nodes)
    .enter().append("g")
    
  var circles = node.append("circle")
      .attr("r", 5)
      //.attr("fill", function(d) { return color(d.group); })
      .call(d3.drag()
          .on("start", dragstarted)
          .on("drag", dragged)
          .on("end", dragended));

  var lables = node.append("text")
      .text(function(d) {
        return d.name;
      })
      .attr('x', 6)
      .attr('y', 3);

  node.append("title")
      .text(function(d) { return d.name; });

  simulation
      .nodes(graph.nodes)
      .on("tick", ticked);

  simulation.force("link")
      .links(graph.links);

  function ticked() {
    link
        .attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });

    node
        .attr("transform", function(d) {
          return "translate(" + d.x + "," + d.y + ")";
        })
  }
});

function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.1).restart();
  d.fx = d.x;
  d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null;
  d.fy = null;
}
});